In [1]:
import tensorflow as tf  # TensorFlow for building and training the neural networks
from tensorflow.keras import layers, Model  # Layers and Model for modular design
import numpy as np  # NumPy for numerical operations
import matplotlib.pyplot as plt  # For visualizing results


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the images to [-1, 1] range
x_train = (x_train / 127.5) - 1.0  # Normalize to [-1, 1]
x_test = (x_test / 127.5) - 1.0

# Resize images to (128, 128) for better resolution (optional)
x_train = tf.image.resize(x_train, (128, 128))
x_test = tf.image.resize(x_test, (128, 128))

# Check dataset shape
print(f"x_train shape: {x_train.shape}")


In [ ]:
y_train_one_hot = tf.keras.utils.to_categorical(y_train, 10)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, 10)

# Check label shape
print(f"y_train shape: {y_train_one_hot.shape}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define Generator
class Generator(tf.keras.Model):
    def __init__(self, num_domains):
        super(Generator, self).__init__()
        self.num_domains = num_domains
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(128, 128, 3)),
            layers.Conv2D(64, 4, 2, padding="same", activation="relu"),
            layers.Conv2D(128, 4, 2, padding="same", activation="relu"),
            layers.Conv2D(256, 4, 2, padding="same", activation="relu"),
            layers.Conv2D(512, 4, 2, padding="same", activation="relu")
        ])
        self.fc = layers.Dense(512, activation="relu")
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(512, 4, 2, padding="same", activation="relu"),
            layers.Conv2DTranspose(256, 4, 2, padding="same", activation="relu"),
            layers.Conv2DTranspose(128, 4, 2, padding="same", activation="relu"),
            layers.Conv2DTranspose(64, 4, 2, padding="same", activation="relu"),
            layers.Conv2D(3, 3, 1, padding="same", activation="tanh")  # Final output layer
        ])

    def call(self, input_image, domain_label):
        x = self.encoder(input_image)
        x = self.fc(x)
        x = tf.concat([x, domain_label], axis=-1)  # Concatenate domain label
        generated_image = self.decoder(x)
        return generated_image

# Define Discriminator
class Discriminator(tf.keras.Model):
    def __init__(self, num_domains):
        super(Discriminator, self).__init__()
        self.num_domains = num_domains
        self.model = tf.keras.Sequential([
            layers.InputLayer(input_shape=(128, 128, 3)),
            layers.Conv2D(64, 4, 2, padding="same", activation="relu"),
            layers.Conv2D(128, 4, 2, padding="same", activation="relu"),
            layers.Conv2D(256, 4, 2, padding="same", activation="relu"),
            layers.Conv2D(512, 4, 2, padding="same", activation="relu"),
            layers.Flatten(),
            layers.Dense(1, activation="sigmoid")  # Output real/fake score
        ])

    def call(self, input_image):
        return self.model(input_image)

# Instantiate models
num_domains = 10  # 10 classes in CIFAR-10
generator = Generator(num_domains)
discriminator = Discriminator(num_domains)


In [ ]:
# Adversarial loss (Binary Cross Entropy)
adversarial_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Domain classification loss (Categorical Crossentropy)
classification_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# L1 loss (for image reconstruction)
l1_loss = tf.keras.losses.MeanAbsoluteError()


In [ ]:
# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)

@tf.function
def train_step(real_images, domain_labels):
    # Generate fake images
    fake_images = generator(real_images, domain_labels)

    # Discriminator loss
    real_output = discriminator(real_images)
    fake_output = discriminator(fake_images)
    real_labels = tf.ones_like(real_output)
    fake_labels = tf.zeros_like(fake_output)

    d_loss_real = adversarial_loss(real_labels, real_output)
    d_loss_fake = adversarial_loss(fake_labels, fake_output)
    d_loss = d_loss_real + d_loss_fake

    # Generator loss
    g_loss = adversarial_loss(real_labels, fake_output) + classification_loss(domain_labels, fake_images)

    # Backpropagation
    gradients_of_generator = tf.gradients(g_loss, generator.trainable_variables)
    gradients_of_discriminator = tf.gradients(d_loss, discriminator.trainable_variables)

    # Update weights
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return g_loss, d_loss


In [ ]:
def generate_image(input_image, target_domain_label):
    generated_image = generator(input_image, target_domain_label)
    return generated_image


In [ ]:
for epoch in range(num_epochs):
    for batch in range(total_batches):
        real_images, domain_labels = next_batch()  # Your batch loading logic here
        g_loss, d_loss = train_step(real_images, domain_labels)

    print(f"Epoch {epoch}, Generator Loss: {g_loss}, Discriminator Loss: {d_loss}")
